<a href="https://colab.research.google.com/github/gsdev42/Medical-question-summarization-/blob/main/BartVsT_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

using T-5


In [ ]:
!pip install sacremoses==0.0.53
!pip install datasets torchvision torchaudio

In [ ]:
!pip install transformers

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install torch

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
sample1="""SUBJECT: CPTII Deficiency
MESSAGE: My sister in Texas was just diagnosed with CPTII deficiency today.
 We do not yet know which variant.
  I was told a year ago following her near death from rhabdo (CK's 150,000+),
   ATN, RF, respiratory compromise, to be tested for a FA metabolic error.
Until today, I really had no idea which test to request.
Her muscle bx was performed a few months ago as she was still ill last year.
Another recurrence of rhado this year prompted the bx (she is retired military).
I don't know which branch of medicine to contact for genetic testing.
From  the NIH website, muscle bx is the most definitive.
I also found a phone number for the Genetics and Rare Diseases Information Center.  Thank you for some direction."""

sample2="""SUBJECT: Customer Service Request
MESSAGE: Hi,
We are 2 sisters and a brother. My brother at age of 24 is detected with catract,
however it is mild so operation is not required. But doctor said, there are chances he has this from Birth.
 However only from past 1.5years he had a little blur vision.
And in my entire family Mom and Dad said no one has had catract from Birth.
 We are a big family, my grapa had 7 brothers and 2 sisters no one from their family
  or my mom has catract from birth or even at early age. My concern is, if i am pregnant
  will my children have any issue like this. I amd my sister, mom, dad we all have got our eyes checked.
   we dont have any issue. I am really tensed as i am expecting. Can you please help me with the answer."""


In [ ]:
ref_summary_1="""Where can I get genetic testing for CPTII deficiency?"""
ref_summary_2="""Who is at risk of developing infantile cataracts?"""

In [ ]:
from datasets import load_dataset
from transformers import pipeline

In [ ]:
summarizer = pipeline(
    task="summarization",
    model="t5-small",
    min_length=20,
    max_length=40,
    truncation=True,
    model_kwargs={"cache_dir": '/Documents/Huggin_Face/'},
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:

t5_summary1 = summarizer(sample1, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
t5_summary2 = summarizer(sample2, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
print("Summary:", t5_summary1)
print("Summary:", t5_summary2)

Summary: muscle bx was performed a few months ago as she was still ill last year . she was diagnosed with CPTII deficiency .
Summary: my brother at age of 24 is detected with catract, but it is mild so operation is not required . but doctor said, there are chances he has this from birth . if i am pregnant will my children have any issue .


using BART

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Importing the model
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig

In [ ]:
# Tokenizer and model loading for bart-large-cnn

tokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [ ]:
# Transmitting the encoded inputs to the model.generate() function
input1 = tokenizer.batch_encode_plus([sample1],return_tensors='pt')
summary_ids_1 =  model.generate(input1['input_ids'], num_beams=4, max_length=150, early_stopping=True)
input2 = tokenizer.batch_encode_plus([sample2],return_tensors='pt')
summary_ids_2 =  model.generate(input2['input_ids'], num_beams=4, max_length=150, early_stopping=True)

In [ ]:
# Decoding and printing the summary
bart_summary1 = tokenizer.decode(summary_ids_1[0], skip_special_tokens=True)
bart_summary2 = tokenizer.decode(summary_ids_2[0], skip_special_tokens=True)
print(bart_summary1)
print(bart_summary2)

My sister in Texas was just diagnosed with CPTII deficiency today. We do not yet know which variant. I don't know which branch of medicine to contact for genetic testing. From the NIH website, muscle bx is the most definitive. I also found a phone number for the Genetics and Rare Diseases Information Center.
My brother at age of 24 is detected with catract,  however it is mild so operation is not required. But doctor said, there are chances he has this from Birth. However only from past 1.5years he had a little blur vision. My concern is, if i am pregnant  will my children have any issue like this.


computing bleu score

In [ ]:
import sacrebleu

reference_summaries = [
    ref_summary_1,
    ref_summary_2

]

generated_summaries_t5 = [
  t5_summary1,
  t5_summary2
]

generated_summaries_bart = [
    bart_summary1,
    bart_summary2
]

# Flatten the reference summaries for sacrebleu
flattened_references = [[ref] for refs in reference_summaries for ref in refs]

# Calculate BLEU for T5
bleu_t5 = sacrebleu.corpus_bleu(generated_summaries_t5, flattened_references)
print(f"T5 BLEU score: {bleu_t5.score}")

# Calculate BLEU for BART
bleu_bart = sacrebleu.corpus_bleu(generated_summaries_bart, flattened_references)
print(f"BART BLEU score: {bleu_bart.score}")


T5 BLEU score: 1.6466642419110007
BART BLEU score: 0.7188461938966488
